**Prompt**: Use pure tensorflow tensor data structure to build a 3 layer neural network (with relu nonlinearity), not using auto differentiation. 

In [1]:
import tensorflow as tf

# Creating Nonlinear Equation, Generating Data

In [10]:
# Generating data of 3 dim input and 2 dim output
n, d = 500, 3
x = tf.random.uniform(minval=-1, maxval=1, shape = (n, d))
x2 = tf.cast(x**2, tf.float32)
weights_true = tf.cast(tf.constant([[5,1,5],[1,2,1]]), tf.float32)
bias_true = tf.cast(tf.constant([1,2]), tf.float32)
y_true = tf.matmul(x2, weights_true, transpose_b=True) + tf.matmul(x,weights_true,transpose_b=True) + bias_true

print(f'x: {x.shape}, weights: {weights_true.shape}, bias: {bias_true.shape}, y: {y_true.shape}')

x: (500, 3), weights: (2, 3), bias: (2,), y: (500, 2)


# Building 3 layer NN with Tensorflow using GradientTape

## Linear Layer
* Randomly initialize weight and bias
* Compute forward pass and gradient descent
* Update learnable parameters

In [11]:
from tensorflow.keras.layers import Layer
class Linear(Layer):
  """y = w.x + b"""

  def __init__(self, units=32):
      super(Linear, self).__init__()
      self.units = units

  def build(self, input_shape):
      self.w = self.add_weight(shape=(input_shape[-1], self.units),
                               initializer='random_normal',
                               trainable=True)
      self.b = self.add_weight(shape=(self.units,),
                               initializer='random_normal',
                               trainable=True)

  def call(self, inputs):
      return tf.matmul(inputs, self.w) + self.b

## Model Class
* Compute forward pass
* Compute backward pass

In [12]:
class MLP(Layer):
    """Simple stack of Linear layers."""

    def __init__(self):
        super(MLP, self).__init__()
        self.linear_1 = Linear(22)
        self.linear_2 = Linear(12)
        self.linear_3 = Linear(2)

    def call(self, inputs):
        x = self.linear_1(inputs)
        x = tf.nn.relu(x)
        x = self.linear_2(x)
        x = tf.nn.relu(x)
        return self.linear_3(x)

# Training/Testing the Model

In [14]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
mse_loss_fn = tf.keras.losses.MeanSquaredError()
loss_metric = tf.keras.metrics.Mean()

dataset = tf.data.Dataset.from_tensor_slices((x, y_true))
dataset = dataset.shuffle(buffer_size=1).batch(30)

epochs = 1000
mlp = MLP()
# Iterate over epochs.
for epoch in range(epochs):

    # Iterate over the batches of the dataset.
    for step, (x_batch, y_batch) in enumerate(dataset):
        with tf.GradientTape() as tape:
            # Forward Pass
            y_pred_batch = mlp(x_batch)
            # Compute loss
            loss = mse_loss_fn(y_batch, y_pred_batch)
            loss += sum(mlp.losses)  # Add regularization loss

        grads = tape.gradient(loss, mlp.trainable_weights) # Use autograd
        optimizer.apply_gradients(zip(grads, mlp.trainable_weights)) # Update learnable parameter

        loss_metric(loss)
    if epoch % 100 == 0:
      print(f'Epoch {epoch}, loss {loss_metric.result()}')

Epoch 0, loss 27.35096549987793
Epoch 100, loss 10.890510559082031
Epoch 200, loss 6.742551803588867
Epoch 300, loss 4.970281600952148
Epoch 400, loss 3.949159622192383
Epoch 500, loss 3.320270538330078
Epoch 600, loss 2.8876240253448486
Epoch 700, loss 2.5639238357543945
Epoch 800, loss 2.296229124069214
Epoch 900, loss 2.0633046627044678


# Plotting Training Data and Prediction

In [15]:
def plot_intereactive_3d(x, y, y_pred=None):
  import plotly.graph_objects as go

  fig = go.Figure()
  fig.add_trace(go.Scatter3d(x = x[:,0],
                    y = x[:,1],
                    z = y.reshape([-1]),
                    opacity=0.5, mode='markers', name='Underlying Function'
                    ))
 
  if y_pred is not None:
    fig.add_trace(go.Scatter3d(x = x[:,0],
                   y = x[:,1],
                   z = y_pred.reshape([-1]),
                   opacity=0.5, mode='markers', name='Predicted Function'
                  ))
    
  fig.update_layout(scene = dict(
                    xaxis_title='X1',
                    yaxis_title='X2',
                    zaxis_title='Y'),
                    width=700,
                    margin=dict(r=20, b=10, l=10, t=10))
  fig.show()

To plot 3 dimensional data, reduce x into 2-dimension using TSNE 

In [17]:
from sklearn.manifold import TSNE
X_reduced = TSNE(n_components=2).fit_transform(x)
y_true_reduced = TSNE(n_components=1).fit_transform(y_true)
y_pred_reduced = TSNE(n_components=1).fit_transform(mlp(x))
print(f'X_reduced: {X_reduced.shape}, y_true_reduced: {y_true_reduced.shape}, y_pred_reduced: {y_pred_reduced.shape}')
plot_intereactive_3d(X_reduced,y_true_reduced,y_pred_reduced)

X_reduced: (500, 2), y_true_reduced: (500, 1), y_pred_reduced: (500, 1)
